# IMPORT

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time, sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "./input"]).decode("utf8"))


# Any results you write to the current directory are saved as output.

# MAIN

In [20]:
# Set of feature extractors
#			 key	  feature class
features1 = {"BoW": BoW()}
features2 = {"cos": CosSim()}
# Set of classifier models
#		   key		  model
models = {"forest": RandomForest}

#TEST
DO_feature1_feature2_model(BoW, CosSim, RandomForest)

Feature1: BoW
Feature2: CosineSimilarity
Model: Random Forest
SPLIT DATA
LOAD TRAIN DATA
TRAIN
FIT CORPUS ON COUNT VECTORIZER
PROCESSING: COMPUTE COS SIM; PROGRESS 7000 / 7000            
 FIT RANDOM FOREST
PREDICTING
RANDOM FOREST PREDICTION IM; PROGRESS 3000 / 3000            

EVALUATING
SCORE ROC-AUC: 0.631259219898
TRAINING ON WHOLE TRAINSET FOR CREATING SUBMISSION FILE
LOAD TRAIN DATA FOR PREDICT AND SUBMIT
LOAD TRAIN DATA
TRAIN
LOAD TEST DATA
FIT CORPUS ON COUNT VECTORIZER
PROCESSING: COMPUTE COS SIM; PROGRESS 10000 / 10000            
 FIT RANDOM FOREST
LOAD TEST DATA
PREDICTING
RANDOM FOREST PREDICTION IM; PROGRESS 10000 / 10000            

Create Submission File


# EVALUATE

In [3]:
def DO_feature1_feature2_model(feature1, feature2, model, fitontest=False, ratio = 0.7):
    feature1_name, feature2_name, model_name = feature1.name, feature2.name, model.name;
    print ("===========================================================")
    print ('\033[1mFeature1: \033[07m\033[04m' + feature1_name + '\033[0m')
    print ('\033[1mFeature2: \033[07m\033[04m' + feature2_name   + '\033[0m')
    print ('\033[1mModel: \033[07m\033[04m' + model_name   + '\033[0m')
    time.sleep(1)  
    # LOAD DATA
    X_train, X_traintest, Y_train, Y_traintest = splitData()
    # SPLIT AND TEST ON TRAIN DATA
        #Train on Train Data (optionallay CountVect also on Test-Set cause unsupervised)
    X1_train = X_train.append(X_traintest) # Train unsupervised with whole Training set
    X2_train = X_train # Train supervised only with Train Split
    f1,f2,cl = Train(X1_train, X2_train, Y_train, feature1, feature2, model, \
                     fitontest = False)
        #Predict and Evaluate
    pred = Predict(f1, f2, cl, X_traintest)
    score = Evaluate(Y_traintest, pred)

    # TRAIN ON TRAIN (WITHOUT SPLOT)
    print("TRAINING ON WHOLE TRAINSET FOR CREATING SUBMISSION FILE")
    XT_train, YT_train = trainData()
    f1,f2,cl = Train(XT_train, XT_train, YT_train, feature1, feature2, model, fitontest = True)
    # PREDICT SUBMIT FILE
    XT, ids = testData()
    pred = Predict(f1,f2, cl, XT)
    # Create Submission File
    PreToSub(ids, pred)

In [21]:
def Train(X1_train, X2_train, Y_train, feature1, feature2, model, fitontest = False):
    # 1st Stage: Question to Matrix
    print("TRAIN")
    f1 = feature1()
    if(fitontest):
        X_test = load_test()
        f1.fit(X1_train.append(X_test)) # This douple-appending is a bit ugly but it works
    else:
        f1.fit(X1_train)
    # 2nd Stage: Matrices of 2 Questions to Feature Input for Classifier
    f2 = feature2()
    X_train = f2.transform(X2_train, f1)
    #Train Classifier
    cl = model()
    cl.fit(X_train, Y_train)
    return f1,f2,cl

def Predict(f1, f2, cl, X_traintest):
    print("PREDICTING")
    X_traintest = f2.transform(X_traintest, f1)
    pred = cl.predict(X_traintest)
    return pred
def Evaluate(Y_true, Y_pred):
    print("EVALUATING")
    score = roc_auc_score(Y_true, Y_pred)
    print("SCORE ROC-AUC: %s" %(score))
    return score
    
def PreToSub(ids, pred):
    print("Create Submission File")
    sub = pd.DataFrame(data={"test_id":ids,"is_duplicate":pred})
    sub.to_csv('sub.csv', columns=['test_id', 'is_duplicate'], index=False)

# PREPROCESSING

In [15]:
Train_Path = './input/df10.csv' # JUST TO TEST FASTER , normally train.csv
Test_Path = './input/df10.csv'

def load_train():
    print("LOAD TRAIN DATA")
    train = pd.read_csv(Train_Path).dropna()
    return train
def load_test():
    print("LOAD TEST DATA")
    test = pd.read_csv(Test_Path).dropna()
    return test
def splitData(ratio = 0.7): # ratio = train_set, between 0-1.0, default 0,7
    print("SPLIT DATA")
    #time.sleep(1)
    train = load_train()
    X_train, X_test, Y_train, Y_test = train_test_split(train.loc[:,["question1", "question2"]],\
                                                        train["is_duplicate"],\
                                                        train_size=0.7, random_state=42)
    return X_train, X_test, Y_train, Y_test
def trainData():
    print("LOAD TRAIN DATA FOR PREDICT AND SUBMIT")
    train = load_train()
    X = train.loc[:, ["question1", "question2"]]
    Y = train.loc[:]["is_duplicate"]
    return X,Y
def testData():
    test = load_test()
    X_test = test.loc[:,["question1", "question2"]]
    X_ids = test.loc[:]["id"]
    return X_test, X_ids

def rprint(str): # Next print overwrites this, eg use for indicate progress
	sys.stdout.write("PROCESSING: " + str + "            \r")
	sys.stdout.flush()

# FEATURES
## 1st Stage: Question to Matrix

In [6]:
class BoW:
    name = "BoW"
    cv = 0

    def fit(self, X):
        corpus = pd.concat([X.question1, X.question2])
        self.cv = CountVectorizer(analyzer='word',min_df = 0,
        max_features=5000, ngram_range=(1,1), preprocessor=None, stop_words=None,
        tokenizer=None)
        print("FIT CORPUS ON COUNT VECTORIZER")
        self.cv.fit(corpus)

    def transform(self, X):
        return self.cv.transform(X)

## 2nd Stage: Matrices of 2 Questions to Feature Input for Classifier

In [7]:
class CosSim:
    name="CosineSimilarity"
    def transform(self, X, cv):
        all_csim = []
        counter = 0
        for index, row in X.iterrows():
            counter += 1
            a = row["question1"]
            b = row["question2"]
            similarity = cosine_similarity(cv.transform([a]), cv.transform([b])).ravel()[0]
            all_csim.append([similarity])
            if (counter%100==0):
                rprint("COMPUTE COS SIM; PROGRESS %d / %d" % (counter,len(X)))
        return all_csim

# classifier

In [8]:
class RandomForest:
    name = "Random Forest"
    forest = RandomForestClassifier(n_estimators = 100)
    def fit(self, X, Y):
        print("\n FIT RANDOM FOREST")
        self.forest.fit(X,Y)
    def predict(self, X):
        print("RANDOM FOREST PREDICTION \n")
        Y = self.forest.predict(X)
        return Y